# BrainStation Capstone: Flight Data and CO2 Emissions

**Author:** Xenel Nazar

**Contact Info:** xenel.nazar@gmail.com

**Submission Date:** Sept 26, 2022

**Notebook:** 1 of 4

Table of Contents:

[Introduction](#Introduction) \
[Data Overview](#Data-Overview) \
[Data Dictionary](#Data-Dictionary) \
[Data Wrangling/Cleaning](#Data-Wrangling/Cleaning) \
[Sources](#Sources) 

# Introduction

Climate change is widely considered as a major concern for many in our generation. The aviation industry accounts for 2.5% of total carbon emissions, and the emissions from the industry have doubled since the mid-1980s ([1](https://ourworldindata.org/co2-emissions-from-aviation)). Recent celebrity scrutiny of utilizing their private jets inefficiently has generated more interest into how everyday people can do their part to limit their CO2 emissions and overall impact to the environment ([2](https://www.buzzfeednews.com/article/lesliefinlay/how-celebrity-private-jet-emissions-affect-environment) & [3](https://www.washingtonpost.com/climate-environment/2022/08/02/taylor-swift-kylie-jenner-private-jet-emissions/)). 

The following capstone project, including all pertinent notebooks, visualizations, and report, will look to generate insights on CO2 emissions generated from commercial flights around the world, and how flyers can ensure that if they do need to fly, they conduct it efficiently.

The plan for the capstone project is to gather our data, execute all required cleaning of the data, run EDA on the data, pre-process the data, and conduct all required modeling and optimization steps to generate key takeaways.

### Import Libraries

We will first load the libraries needed to assist with Data Wrangling and Data Cleaning to help prepare for the next steps in EDA, as well as Modeling. 

In [ ]:
# import libraries
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import matplotlib.pyplot as plt
import plotly.express as px
from geopy import distance
from geopy import Point
from datetime import timedelta

# Data Overview

The data we will use was scraped Barking Data, a web data miner ([4](https://www.barkingdata.com/)). Barking Data utilized flight data scrapped from Google Flights, which provides key details including CO2 Emissions per trip ([5](https://www.google.com/flights/)). Barking Data hosted a sample of their data containing close to a million rows in Kaggle ([6](https://www.kaggle.com/datasets/polartech/flight-data-with-1-million-or-more-records)). 

We will now load the data, which is currently stored in a Google Drive instead of directly from Kaggle, to help eliminate Kaggle authentication requirements.

In [ ]:
# import zip file from google drive into dataframe
url = 'https://drive.google.com/file/d/1d988nLtyFEEba6ksIm7islAgI-bQ9DXj/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df = pd.read_csv(dwn_url, compression='zip')

In [ ]:
# verify dataframe
df.head()

In [ ]:
print('\Data Shape:', df.shape)
print('\nNumber of rows:', df.shape[0])
print('\nNumber of columns:', df.shape[1])

The dataset contains 998,866 rows and 18 columns.

# Data Dictionary

In [ ]:
# info on dataset
df.info()

Barking Data did not include a data dictionary for us to utilize, but from the columns and viewing the contents of the data, we can see that the data contains the following:


Origin Airport Information:
- `from_airport_code` - Origin Airport Code
- `from_country` - Origin Airport Country


Destination Airport Information:
- `dest_airport_code` - Destination Airport Code
- `dest_airport_country` - Destination Airport Country


Aircraft Information:
- `aircraft_type` - Aircraft Type(s)


Airline Information:
- `airline_number` - Airline Number
- `airline_name` - Airline Name(s)


Flight/Trip Information:
- `flight_number` - Flight Number
- `departure_time` - Departure Time
- `arrival_time` - Arrival Time
- `duration` - Duration of Flight/Trip
- `stops` - Number of stops to Destination
- `price` - Price of Flight/Trip
- `currency` - Currency of Price (USD)
- `co2_emissions` - CO2 Emissions generated for Flight/Trip


CO2 Emissions Comparison Metrics:
- `avg_co2_emission_for_this_route` - Average CO2 Emission for the Flight/Trip
- `co2_percentage` - Calculation based on CO2 Emissions and Average CO2 Emissions


Web Scrape Info:
- `scan_date` - Web Scrape Date


# Data Wrangling/Cleaning

We will now review the data in the dataset and perform any necessary cleaning prior to data pre-processing and modeling.

### Review Date Columns

In [ ]:
df.info()

We can see that the data is not in the proper datetime data type. We will need to change all date columns to datetime data type. 

In [ ]:
# change date columns to datetime format.
df['departure_time'] = pd.to_datetime(df['departure_time'])
df['arrival_time'] = pd.to_datetime(df['arrival_time'])
df['scan_date'] = pd.to_datetime(df['scan_date'])

In [ ]:
# verify
df.info()

#### Verify Earliest and Latest Date Info

In [ ]:
# Review Departure Date Info
print('The earliest Departure Date is: ',df['departure_time'].min())
print('The latest Departime Date is: ',df['departure_time'].max())

The departure dates are from the period of April 30,2022 to August 28, 2022. 

In [ ]:
# Review Arrival Date Info
print('The earliest Arrival Date is: ',df['arrival_time'].min())
print('The latest Arrival Date is: ',df['arrival_time'].max())

The arrival dates are from the period of April 30,2022 to September 2, 2022.

In [ ]:
# Review Scan Date Info
print('The earliest Scan Date is: ',df['scan_date'].min())
print('The latest Scan Date is: ',df['scan_date'].max())

Scan date only lists one value, the date the web scrape/mine was done on April 29th, 2022. We can disregard this column as it will not be a factor for EDA and modelling.

In [ ]:
# drop scan_date column
df =df.drop(['scan_date'], axis = 1)

In [ ]:
# verify
df.info()

One factor we have to keep in mind is that the dates listed to do not account for timezones at the destination and arrival destinations. We will account for this later in the notebook to standardize to the UTC/GMT timezone for all locations.

### Verify Duplicates

We can now look at the duplicates in the dataset.

In [ ]:
x = df.duplicated().sum()
y = round((x/df.shape[0])*100,2)

print(f"Total number of duplicate rows:",x)
print(f"The total number of duplicate rows equates to:", y, "%")

There are a total of 51,197 rows that are duplicates which equates to 5.13%. Since these are duplicate rows we can remove these from the dataset. 

In [ ]:
# drop duplicate rows
df = df.drop_duplicates().copy()

In [ ]:
# Verify Duplicates
x = df.duplicated().sum()
y = round((x/df.shape[0])*100,2)

print(f"Total number of duplicate rows:",x)
print(f"The total number of duplicate rows equates to:", y, "%")

### Verify Null Values

We can now look at the null values in the dataset.

In [ ]:
df.info()

We can see that we are missing info in:
- `co2_emissions`
- `price`
- `aircraft_type`

In [ ]:
x = df['co2_emissions'].isnull().sum()
y = round((x/df.shape[0])*100,2)
print(f"Total number of co2_emissions null values:",x)
print(f"The total number of null equates to:", y, "%")

We can see that there are 4,671 rows with missing values for CO2 emissions. We can impute the average CO2 emisssions using the `avg_co2_emissions_for_this_route` column. 

In [ ]:
df['co2_emissions'].fillna(df['avg_co2_emission_for_this_route'], inplace=True)

In [ ]:
# verify
x = df['co2_emissions'].isnull().sum()
y = round((x/df.shape[0])*100,2)
print(f"Total number of co2_emissions null values:",x)
print(f"The total number of null equates to:", y, "%")

Around 4,145 rows or about 0.43% of the total dataset was able to be filled using the `avg_co2_emission_for_this_route` column.

We can now look at the `price` column.

In [ ]:
x = df['price'].isnull().sum()
y = round((x/df.shape[0])*100,2)
print(f"Total number of price null values:",x)
print(f"The total number of null equates to:", y, "%")

There are a total of 1,351 rows with missing values or 0.14% of the dataset.

We can not impute the price column based on an average, as prices are based on the time of booking, as well as other factors including pricing for route, pricing from airline, the number of stops, etc. 

We can now also look at the `aircraft_type`. 

In [ ]:
x = df['aircraft_type'].isnull().sum()
y = round((x/df.shape[0])*100,2)
print(f"Total number of aircraft type null values:",x)
print(f"The total number of null equates to:", y, "%")

The total number of rows missing `aircraft_type` values is 13,885 rows, which equates to 1.47% of the dataset. 

Imputing the values will need actual flight records in case there was a change to the aircraft used prior to the flight.

Since the missing values equate to a small portion of the dataset, removing the values for `price` and `aircraft_type` will not play a significant factor. 

In [ ]:
# drop null values
df.dropna(inplace=True)

In [ ]:
# reset index
df.reset_index(drop = True, inplace = True)

In [ ]:
# verify
df.info()

After dropping the null values, we are now left with 850,735 columns in the dataset.

### Review Airport Codes

We can now review the Airport Codes in the dataset.

In [ ]:
count_unique = df['from_airport_code'].nunique()
print('# Number of Origin Airport Codes:', count_unique) 
print('Origin Airport Codes:')
df['from_airport_code'].unique()


There are a total of 30 Origin Airport Codes in the dataset. 

In [ ]:
count_unique = df['dest_airport_code'].nunique()
print('# Number of Destination Airport Codes:', count_unique) 
print('Destination Airport Codes:')
df['dest_airport_code'].unique()

There are a total of 77 Destination airport codes in the dataset. 

To help with better interpretation, we can include additional information for each airport based on the aiport code.

#### Add Airport & UTC Timezone Adjustment Information

We can use information pulled from the airportcodes.aero website ([7](https://airportcodes.aero/)) to pull Airport Names, as well as Airport City Information. We will also include timezone information pulled from the timeanddate.com website ([8](https://www.timeanddate.com/)) to help standardize departure and arrival times. 

In [ ]:
# Load Airport Information
airport_information = {
    'Airport_Code':[ 'AEP', 'CAN', 'MUC', 'HND', 'LIM', 'CPT', 'LGW', 'IAH', 'HGH', 'SYD', 'SZX', 'FRA', 'NBO', 'LIS', 'MAD', 'MAN', 'SEA', 'CTU', 'MEL', 'PVG', 'DEL', 'MEX', 'FCO', 'TPE', 'ATL', 'FLL', 'XIY', 'VIE', 'BOG', 'ATH', 'KUL', 'DOH', 'ARN', 'ORD', 'SFO', 'VCP', 'GRU', 'CPH', 'BOM', 'CMN', 'SVO', 'BKK', 'LAX', 'JFK', 'CGO', 'BRU', 'DUB', 'CGK', 'OSL', 'SIN', 'SAW', 'PHX', 'ZRH', 'PEK', 'YYZ', 'ADD', 'BLR', 'AMS', 'DME', 'IST', 'MCO', 'ALG', 'CNF', 'CAI', 'MXP', 'PTY', 'JNB', 'DXB', 'CLT', 'SGN', 'SCL', 'CDG', 'NRT', 'MNL', 'ICN', 'LHR', 'MIA', 
                    'SHA'],
    'Airport_Name':["Aeroparque Internacional Jorge Newbery","Guangzhou Baiyun International Airport","Munich Airport","Tokyo-Haneda International Airport","Jorge Chavez International Airport","Cape Town International Airport","London-Gatwick Airport","George Bush Intercontinental Airport","Hangzhou Xiaoshan International Airport","Sydney Airport","Shenzhen Bao'an International Airport","Frankfurt Airport","Jomo Kenyatta International Airport","Humberto Delgado Airport","Adolfo Suárez Madrid–Barajas Airport","Manchester Airport","Seattle-Tacoma International Airport","Chengdu Shuangliu International Airport","Melbourne Airport","Shanghai Pudong International Airport","Indira Gandhi International Airport","Mexico City International Airport","Leonardo da Vinci–Fiumicino Airport","Taoyuan International Airport","Hartsfield–Jackson Atlanta International Airport","Fort Lauderdale–Hollywood International Airport","Xi'an Xianyang International Airport","Vienna International Airport","El Dorado International Airport","Athens International Airport","Kuala Lumpur International Airport","Hamad International Airport","Stockholm-Arlanda Airport","Chicago O'Hare International Airport","San Francisco International Airport","Viracopos International Airport","São Paulo Guarulhos International Airport","Copenhagen Airport","Chhatrapati Shivaji Maharaj International Airport","Mohammed V International Airport","Sheremetyevo Alexander S. Pushkin International Airport","Suvarnabhumi Airport","Los Angeles International Airport","John F. Kennedy International Airport","Zhengzhou Xinzheng International Airport","Brussels Airport","Dublin Airport","Soekarno–Hatta International Airport","Oslo Airport","Singapore Changi Airport","Sabiha Gokcen International Airport","Phoenix Sky Harbor International Airport","Zurich Airport","Beijing Capital International Airport","Toronto Pearson International Airport","Addis Ababa Bole International Airport","Kempegowda International Airport","Amsterdam Airport Schiphol","Moscow Domodedovo Airport","Ataturk International Airport","Orlando International Airport","Houari Boumediene Airport","Tancredo Neves International Airport","Cairo International Airport","Milan Malpensa International Airport","Tocumen International Airport","O. R. Tambo International Airport","Dubai International Airport","Charlotte Douglas International Airport","Tan Son Nhat International Airport","Arturo Merino Benitez International Airport","Paris Charles de Gaulle Airport","Tokyo-Narita International Airport","Ninoy Aquino International Airport","Incheon International Airport","London-Heathrow Airport","Miami International Airport",
                    "Shanghai Hongqiao International Airport"],
    'Airport_City':["Buenos Aires","Guangzhou","Munich","Tokyo","Lima","Cape Town","London","Houston","Hangzhou","Sydney","Shenzhen","Frankfurt","Nairobi","Lisbon","Madrid","Manchester","Seattle","Chengdu","Melbourne","Shanghai","Delhi","Mexico City","Fiumicino","Taipei","Atlanta","Miami","Xi'an","Vienna","Bogota","Athens","Kuala Lumpur","Doha","Stockholm","Chicago","San Francisco","Sao Paulo","Sao Paulo","Copenhagen","Mumbai","Casablanca","Moscow","Bangkok","Los Angeles","New York","Zhengzhou","Brussels","Dublin","Jakarta","Oslo","Singapore","Istanbul","Phoenix","Zurich","Beijing","Toronto","Addis Ababa","Bangalore","Amsterdam","Moscow","Istanbul","Orlando","Algiers","Confins","Cairo","Milan","Panama City","Johannesburg","Dubai","Charlotte","Ho Chi Minh City","Santiago","Paris","Tokyo","Manila","Seoul","London","Miami",
                    "Shanghai"],
    'Airport_Timezone_Adj_Hours':[-3,8,1,9,-5,2,0,-6,8,10,8,1,3,0,1,0,-8,8,10,8,5,-6,1,8,-5,-5,8,1,-5,2,8,3,1,-6,-8,-3,-3,1,5,1,3,7,-8,-5,8,1,0,7,1,8,3,
                                  -7,1,8,-5,3,5,1,3,3,-5,1,-3,2,1,-5,2,4,-5,7,-4,1,9,8,9,0,-5,8,],
    'Airport_Timezone_Adj_Mins' :[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,],
    'Airport_Lat':[-34.5577389000333,23.3961737465643,48.3511109794145,35.5494804083177,-12.0237588747816,-33.9707321566645,51.1537966691745,29.9903964365879,30.2361803134082,-33.9498118844249,22.6370359336888,50.0464385013313,-1.33117001961051,38.7762130903902,40.4984633725195,53.3558534051921,47.4482057538999,30.5476456103965,-33.1651454983468,31.1446043202944,28.5563885500954,19.4363291225696,41.8037152944108,25.0764191795815,33.6422099998983,26.0744078477887,34.4373233461419,48.1128273780094,4.73430379613192,37.9357417891236,2.74306831574282,25.2864324924182,59.6501703931625,41.9805351962813,37.6169339633645,-23.0079482257404,-23.4301793176096,55.617757591627,19.09381604546,33.3700241410235,55.9737862626875,13.6902976061099,33.9502755591614,40.6415799338128,34.5255356432286,50.9016468405679,53.426948602434,-6.11691990670312,60.1980000535908,1.36466689225704,40.9055980244643,33.4354728132571,47.4614632409321,40.0800871350849,43.6778882873949,8.98655469920092,13.1996798857919,52.3131357641106,55.4118246221447,40.9825745365632,28.418020478555,36.6982358241641,-19.6341898897342,30.1121841725708,45.6302725501154,9.063255632165,-26.136354934708,25.2534558849676,35.2144638755156,10.8186527789434,-33.3968058771658,49.0088399679511,35.7723087585301,14.5122946629056,37.4618712161197,51.4700089131301,25.7958143253594,31.1923558264639],
    'Airport_Lon':[-58.4166440211707,113.308527797486,11.776874580731,139.779881500914,-77.1116283062124,18.6049811878035,-0.181665934543478,-95.3363750061318,120.439470055245,151.182320094158,113.815078066792,8.55275159050158,36.9240548993851,-9.13382237936812,-3.56679557629829,-2.27798454703279,-122.30797122374,103.92756068242,151.699213085184,121.808747972798,77.1006659010293,-99.0717151831705,12.2523434092231,121.232181901905,-84.4509584649548,-80.1503125235807,108.757876568594,16.5761147130645,-74.0809861417985,23.9485819051683,101.707313313655,51.5931790022843,17.9258703687482,-87.9082146968791,-122.39850911226,-47.1369462295634,-46.4733755958963,12.6763197130793,72.8723824505524,-7.58521884676506,37.4131359997803,100.750506909683,-118.386900678162,-73.7775911312842,113.846703468596,4.48710410752413,-6.24856139829551,106.670661181529,11.1025491719827,103.991884849302,29.3173752822154,-112.011164699045,8.5549777283269,116.603702184045,-79.6243690907351,38.7832924789539,77.7055412975771,4.77616403180424,37.9047240882624,28.8099981343228,-81.3035121008196,3.20805280727182,-43.9654174748631,31.4029415288509,8.72609932649832,-79.3901665795341,28.2414677631063,55.3661555954439,-80.9476042938708,106.659500414364,-70.7931920238099,2.55316806685689,140.393418726289,121.016336336393,126.441974742097,-0.454327688131043,-80.2869650714547,121.334887083881]
}

We can place the information into a dataframe to review.

In [ ]:
# Load Airport Information to a dataframe
airport_df = pd.DataFrame(airport_information)
#verify
airport_df

We can merge the data to each of the Origin and Destination Airport Codes.

In [ ]:
# Merge Airport Information to Origin Airport Codes
df = df.merge (airport_df, left_on='from_airport_code', right_on='Airport_Code', how='left')

In [ ]:
# verify
df.head()

We will now rename the columns to identify the Origin Airport Information.

In [ ]:
# rename columns for simplicity
df.rename(columns = {'Airport_City':'from_Airport_City',
                     'Airport_Name':'from_Airport_Name',
                     'Airport_Timezone_Adj_Hours':'from_Airport_Timezone_Adj_Hours',
                     'Airport_Timezone_Adj_Mins':'from_Airport_Timezone_Adj_Mins',
                     'Airport_Lat':'from_Airport_Lat',
                     'Airport_Lon': 'from_Airport_Lon'
                    }, inplace = True)

In [ ]:
# verify
df.head()

We can drop the duplicate `Airport_Code` column from the dataset.

In [ ]:
# drop duplicate Airport_code column
df =df.drop(['Airport_Code'], axis = 1)

In [ ]:
# verify
df.head()

We can now do the same process for the Destination Airport Codes.

In [ ]:
# Merge Airport Information to Destination Airport Codes
df = df.merge (airport_df, left_on='dest_airport_code', right_on='Airport_Code', how='left')

In [ ]:
# verify
df.head()

We will now rename the columns to identify the Destination Airport Information.

In [ ]:
# rename columns for simplicity
df.rename(columns = {'Airport_City':'dest_Airport_City',
                     'Airport_Name':'dest_Airport_Name',
                     'Airport_Timezone_Adj_Hours':'dest_Airport_Timezone_Adj_Hours',
                     'Airport_Timezone_Adj_Mins':'dest_Airport_Timezone_Adj_Mins',
                     'Airport_Lat':'dest_Airport_Lat',
                     'Airport_Lon': 'dest_Airport_Lon'
                    }, inplace = True)

In [ ]:
# verify
df.head()

We can drop the duplicate `Airport_Code` column from the dataset.

In [ ]:
# drop duplicate Airport_code column
df =df.drop(['Airport_Code'], axis = 1)

In [ ]:
# verify
df.head()

To help consolidate the origin and destination columns, we will reorganize the columns into a new dataframe for simplicity. 

In [ ]:
df2 = df[['from_airport_code','from_Airport_Name','from_Airport_City','from_country'
                      ,'from_Airport_Timezone_Adj_Hours','from_Airport_Timezone_Adj_Mins', 'from_Airport_Lat','from_Airport_Lon'
                     ,'dest_airport_code','dest_Airport_Name','dest_Airport_City'
                     ,'dest_country','dest_Airport_Timezone_Adj_Hours','dest_Airport_Timezone_Adj_Mins'
                     ,'dest_Airport_Lat','dest_Airport_Lon'
                     ,'aircraft_type','airline_number','airline_name','flight_number'
                     ,'departure_time','arrival_time','duration','stops','price','currency'
                     ,'co2_emissions','avg_co2_emission_for_this_route','co2_percentage'
                    ]]

In [ ]:
df2.info()

### Calculate Distance

Using the newly added Latitude and Longitude points for each of the airport codes, we can now calculate the distance between them.

We will first create a coordinates column for each airport, using their Latitude and Longitude.

In [ ]:
# Create Origin Coordinates
df2['from_airport_coordinates'] = df2.apply(lambda row: Point(latitude=row['from_Airport_Lat']
                                                              , longitude=row['from_Airport_Lon'])
                                           , axis=1)

In [ ]:
# Create Destination Coordinates
df2['dest_airport_coordinates'] = df2.apply(lambda row: Point(latitude=row['dest_Airport_Lat']
                                                              , longitude=row['dest_Airport_Lon'])
                                           , axis=1)

In [ ]:
# verify
df2.head()

We can calculate the distance between the coordinates by calculating the great circle measurement. The great circle measurement accounts for the spherical shape of the earth and is what is used by ships and aircraft to minimize distance and save time and money ([9](https://transportgeography.org/contents/chapter1/transportation-and-space/great-circle-distance/)). 

In [ ]:
# Calculate Great Circle Distance in km
df2['Distance_km'] = df2.apply(lambda row: 
                              distance.great_circle(row['from_airport_coordinates']
                                , row['dest_airport_coordinates']).km
                              , axis=1).round(decimals=2)

In [ ]:
# verify
df2.head()

### Departure & Arrival Time Adjustment

We can now standardize the departure and arrival times for each trip, based on the newly imported adjustment columns:
- `from_Airport_Timezone_Adj_Hours`
- `from_Airport_Timezone_Adj_Mins`
- `dest_Airport_Timezone_Adj_Hours`
- `dest_Airport_Timezone_Adj_Mins`

We will first standardize the departure time.

In [ ]:
# Standardize Departure Time to UTC
df2['departure_time_utc'] = pd.to_datetime(df2['departure_time'] ) + pd.to_timedelta(df2['from_Airport_Timezone_Adj_Hours'],unit='hours') +pd.to_timedelta(df2['from_Airport_Timezone_Adj_Mins'],unit='minutes')

In [ ]:
#verify
selection = df2[['departure_time'
                 , 'from_Airport_Timezone_Adj_Hours'
                 , 'from_Airport_Timezone_Adj_Mins','departure_time_utc' ]]
selection

In [ ]:
# Standardize Arrival Time to UTC
df2['arrival_time_utc'] = pd.to_datetime(df2['arrival_time'] ) + pd.to_timedelta(df2['dest_Airport_Timezone_Adj_Hours'],unit='hours') +pd.to_timedelta(df2['dest_Airport_Timezone_Adj_Mins'],unit='minutes')

In [ ]:
#verify
selection = df2[['arrival_time'
                 , 'dest_Airport_Timezone_Adj_Hours'
                 , 'dest_Airport_Timezone_Adj_Mins','arrival_time_utc' ]]
selection

We can now drop the original departure and arrival times, as well as the adjustment columns.

In [ ]:
# drop departure times and adjustment columns
df2 =df2.drop(['departure_time','from_Airport_Timezone_Adj_Hours','from_Airport_Timezone_Adj_Mins'], axis = 1)

In [ ]:
# drop arrival times and adjustment columns
df2 =df2.drop(['arrival_time','dest_Airport_Timezone_Adj_Hours','dest_Airport_Timezone_Adj_Mins'], axis = 1)

In [ ]:
# verify
list(df2.columns)

### Review Flight Number

In [ ]:
df2['flight_number'].value_counts()

We can see that the flight numbers are repeated after each '|', likely based on the number of stops taken on the route (e.g., 2 stops taken *FLIGHT#* | *FLIGHT#* | *FLIGHT#*).

We can pull the first flight number to simplify the flight number details.

In [ ]:
# Pull First Flight Number
df2['flight_number'] = df2['flight_number'].str.split('|', expand=True)[0]

In [ ]:
# Vefify
df2['flight_number'].value_counts(normalize=True)

The flight numbers now only list a single flight number for each trip/route.

### Review Aircraft Type

In [ ]:
df2['aircraft_type'].value_counts()

Similar to flight numbers, the dataset includes aircraft types to be used for each stop taken (e.g., 1 stop taken *Aircraft 1* | *Aircraft 2*).
We will split the column into each respective by the '|' character.

In [ ]:
# split aircraft_type column
df_aircraft = df2['aircraft_type'].str.split('|',expand=True)
# add column names
df_aircraft.columns = [ 'aircraft_type_' + str(i + 1) for i in range(len(df_aircraft.columns)) ]

In [ ]:
# verify
df_aircraft

We can see a lot of null values in columns `aircraft_type_3` to `aircraft_type_7` as the trips ended and did not take any further stops to their final destination airport.

We can fill in the missing values by the last Aircraft Type listed.

In [ ]:
# Fill in null values with last Aircraft Type used
df_aircraft = df_aircraft.ffill(axis = 1)

In [ ]:
# verify
df_aircraft

We will now add the Aircraft data to our dataframe.

In [ ]:
# Add Aircraft data to dataframe
df2 = pd.concat([df2, df_aircraft], axis=1)

In [ ]:
# verify
df2.head()

We can now drop the `aircraft_type` column.

In [ ]:
# drop aircraft_type column
df2 =df2.drop(['aircraft_type'], axis = 1)

In [ ]:
# verify
df2.head()

### Review Airline Name

We will now look at the `airline_name` column. 

In [ ]:
df2['airline_name'].value_counts()

We can see that the Airline Names are contained in square brackets, which we can remove.

In [ ]:
# remove square brackets from airline_name column
df2['airline_name'] = df2['airline_name'].str.strip('[]')

In [ ]:
# verify
df2['airline_name'].value_counts()

We can now conduct the same exercise done in Aircraft Types to split the Airlines listed by the '|' character. 

In [ ]:
# split airline_name column
df_airline = df2['airline_name'].str.split('|',expand=True)
# add column names
df_airline.columns = [ 'airline_' + str(i + 1) for i in range(len(df_airline.columns)) ]

In [ ]:
# verify
df_airline

Similar to aircraft types, the subsequent airline columns do not contain values, as the trip to the final destination ended. 

We can impute the last Airline listed to the subsequent columns.

In [ ]:
# Fill empty airline columns with last Airline listed
df_airline = df_airline.ffill(axis = 1)

In [ ]:
# verify
df_airline

We will now add the split airline information to the dataframe.

In [ ]:
# Add Airline Info to dataframe
df2 = pd.concat([df2, df_airline], axis=1)

In [ ]:
# verify
df2.head()

We can now drop the original `airline_name` column.

In [ ]:
# drop airline_name column
df2 =df2.drop(['airline_name'], axis = 1)

In [ ]:
# verify
df2.head()

### Review Currency

We will now review the `currency` column.

In [ ]:
df2['currency'].value_counts(normalize=True)

The only currency listed in the dataset is USD, so we can drop this column from the dataset, as this does not provide any additional information for us.

In [ ]:
# drop currency column
df2 =df2.drop(['currency'], axis = 1)

In [ ]:
# verify
df2.head()

### Review Airline Number

We will now review the `airline_number` column.

In [ ]:
df2['airline_number'].value_counts(normalize=True)

Close to 63% of the rows list 'multi' as the airline number and the balance listing the airline number for the respective Airline (e.g., LH - Lufthansa) ([10](https://www.bts.gov/topics/airlines-and-airports/airline-codes)).

As we have more detailed information listed in the airline columns, we can remove the `airline_number` column.

In [ ]:
# drop airline_number column
df2 =df2.drop(['airline_number'], axis = 1)

In [ ]:
# verify
list(df2.columns)

### Review avg_co2_for_this_route

We will now look at the `avg_co2_emission_for_this_route` column.

In [ ]:
df2['avg_co2_emission_for_this_route'].value_counts(normalize=True)

As this just details the average CO2 emissions for a route, we can drop the column as it will not provide any other details we can use. 

In [ ]:
df2 =df2.drop(['avg_co2_emission_for_this_route'], axis = 1)

In [ ]:
# verify
df2.head()

### Review co2_percentage

We will now look at the `co2_percentage` column.

In [ ]:
df2['co2_percentage'].value_counts(normalize=True)

As this is just a calculated field between the CO2 emissions, as well as the average for a particular route, we can drop the column.

In [ ]:
df2 =df2.drop(['co2_percentage'], axis = 1)

In [ ]:
# verify
list(df2.columns)

### Review co2_emissions

We can now look at the `co2_emissions` for more insight. 

In [ ]:
df2['co2_emissions'].value_counts(normalize=True)

We can see that for the `co2_emissions` column, it does not detail the units (e.g., lbs, kg) for the particular trip/route. 

We will look at a route and verify the actual CO2 emissions to understand what is actually listed in the column. We can look at the Toronto to London Heathrow route (i.e., YYZ-LHR) in a new dataframe.

In [ ]:
# generate sample dataframe for trips from YYZ to LHR
rslts_df = df2[(df2['from_airport_code'] == 'YYZ') 
               & (df2['dest_airport_code'] == 'LHR') 
               & (df2['stops'] == 0)]

We can filter on the new dataframe just to see the CO2 emissions.

In [ ]:
# view origin airport details, destination airport details, and CO2 emissions
rslts_df[['from_airport_code','from_Airport_City','dest_airport_code','dest_Airport_City','co2_emissions']]

We can also look at the average CO2 emissions for this route.

In [ ]:
# mean of YYZ-LHR route
print("Average CO2 emissions for YYZ-LHR route", rslts_df['co2_emissions'].describe()['mean'])

The average CO2 emissions for the YYZ-LHR route is 570,142.

Based on ICAO emissions calculator for YYZ to LHR, average ~670lbs, so we can assume the co2_emissions column is listed in LBS, but will need to correct it by dividing by 1,000 ([11](https://www.icao.int/environmental-protection/Carbonoffset/Pages/default.aspx)). 

In [ ]:
# correct co2_emissions
df2['co2_emissions'] = df2['co2_emissions'].div(1000)

In [ ]:
# verify
df2['co2_emissions'].head()

In [ ]:
# Emissions
fig = px.histogram(df2, x="co2_emissions",
                   opacity=0.8,
                   color_discrete_sequence=['cornflowerblue']
                   )
fig.update_layout( title='CO2 Emissions Overview',
                  xaxis_title='CO2 Emissions', yaxis_title='Count')
fig.show()
# export Graph
fig.write_html("visualizations/CO2_Emissions_Overview.html")

We can see that CO2 emissions are right skewed, with what looks to be two distibutions, one from 0-500lbs, and another for emissions greater than 500lbs. 

There likely is an underlying feature which explains this case that would lead to us seeing this in the data.

### Calculated Field KM/LBS

We can also look at the relationship between the distance and CO2 emissions, and use it as a calculated field for later.

In [ ]:
# Generate KM/LBS
df2['KM/LBS'] = df2['Distance_km'] / df2['co2_emissions']

In [ ]:
# Verify
df2.head()

In [ ]:
# KM/LBS
fig = px.histogram(df2, x="KM/LBS",
                   opacity=0.8,
                   color_discrete_sequence=['cornflowerblue']
                   )
fig.update_layout( title='KM/LBS Overview',
                  xaxis_title='KM/LBS', yaxis_title='Count')
fig.show()
# export Graph
fig.write_html("visualizations/KM-LBS_Overview.html")

We can see that there is a binormal distribution, with two peaks one from 0 to 4 and another >4. This also implies that for each lb generated flying, one would ideally want to maximize their distance flown to limit their CO2 emissions impact on the environment. We can classify these two by mapping them to Low Utilization or "0" (0-4 km/lbs), and High Utilization "1" (>4 km/lbs). 

In [ ]:
df2['KM/LBS_Classification'] = np.where(df2['KM/LBS'] >4, 1, 0)

In [ ]:
# Check distribution of values
rvw = df2['KM/LBS_Classification'].value_counts(normalize=True)
print(x)

In [ ]:
# KM/LBS_Classification
fig = px.bar(rvw,
                   opacity=0.8,
                   color_discrete_sequence=['cornflowerblue']
                   )
fig.update_layout( title='KM/LBS Classification Breakdown (High Utilization-1 & Low Utilization-0)',
                  xaxis_title='KM/LBS Classification', yaxis_title='Value')
fig.show()
# export Graph
fig.write_html("visualizations/KM-LBS_Classification_Overview.html")

We can see that almost 80% of the classification would be high utilization, but close to 14% of all trips are not efficient in maximizing their KM flown per CO2 lbs generated.

### Routes

To help with interpretability, we can create routes to understand quickly the Origin and Destination Cities.

In [ ]:
# generate routes
df2['Route'] = [str(x) + '-' + y for x, y in zip(df2['from_Airport_City']
                                                 , df2['dest_Airport_City'])]

In [ ]:
# verify
df2.head()

We will now reorganize and export the dataframe to assist with generating visualizations.

In [ ]:
flight_data_edit = df2[["Route","from_airport_code","from_Airport_Name","from_Airport_City","from_country","from_Airport_Lat","from_Airport_Lon","from_airport_coordinates","dest_airport_code","dest_Airport_Name","dest_Airport_City","dest_country","dest_Airport_Lat","dest_Airport_Lon","dest_airport_coordinates","flight_number","aircraft_type_1","aircraft_type_2","aircraft_type_3","aircraft_type_4","aircraft_type_5","aircraft_type_6","aircraft_type_7","airline_1","airline_2","airline_3","airline_4","airline_5","departure_time_utc","arrival_time_utc","duration","stops","price","co2_emissions","Distance_km","KM/LBS","KM/LBS_Classification",
                    ]]

In [ ]:
# verify
print("Previous Dataframe Shape", df2.shape)
print("New Dataframe Shape", flight_data_edit.shape)

We will now export the dataframe as a csv to our data folder for simplicity and use in subsequent notebooks.

In [ ]:
flight_data_edit.to_csv('data/flight_data_edit.csv', index=False)

# Sources

(1) - https://ourworldindata.org/co2-emissions-from-aviation

(2) - https://www.buzzfeednews.com/article/lesliefinlay/how-celebrity-private-jet-emissions-affect-environment

(3) - https://www.washingtonpost.com/climate-environment/2022/08/02/taylor-swift-kylie-jenner-private-jet-emissions/

(4) - https://www.barkingdata.com/

(5) - https://www.google.com/flights/

(6) - https://www.kaggle.com/datasets/polartech/flight-data-with-1-million-or-more-records

(7) - https://airportcodes.aero/

(8) - https://www.timeanddate.com/

(9) - https://www.bts.gov/topics/airlines-and-airports/airline-codes

(10) - https://www.icao.int/environmental-protection/Carbonoffset/Pages/default.aspx

(11) - https://transportgeography.org/contents/chapter1/transportation-and-space/great-circle-distance/
